## Calibrating an sCMOS camera 
This notebook explains how to calibrate an sCMOS camera, i.e. how to measure the offset, variance, gain and relative quantum efficiency of each pixel in the camera. The gain is determined under the assumption that the photo-electrons (e-) detected per well are Poisson distributed. This distribution has the property that it's mean is equal to it's variance.

Note:
* The Poisson assumption is not valid for an EMCCD camera due to the excess noise added by the amplification stage. However to a pretty good approximation, the variance scales as twice the mean so you'll get basically the right answer by multiplying the measured gain by 2.
* This calibration does not tell you anything about the absolute quantum efficiency of the camera, despite the occasional claim to the contrary.


### Configuring the directory
Create an empty directory somewhere on your computer and tell Python to go to that directory.

In [ ]:
import os
os.chdir("/home/hbabcock/Data/storm_analysis/jy_testing/")
print(os.getcwd())

Generate sample data for analysis. 

Here we are just going to generate a bunch of simulated movies. In practice you would do this by taking one movie with the camera shutter closed and the other movies at different, and more or less constant illuminations. One way do this is using the bright-field lamp on the microscope and a low magnification objective (like a 10x) with no sample.

Note also that in order to measure the variance and gain to better than 1% you need to take at least 20k frames for each light intensity.

In [ ]:
import numpy
import numpy.random
import tifffile

n_frames = 4000
x_size = 60
y_size = 50
gain = numpy.random.normal(loc = 2.0, scale = 0.1, size = (x_size, y_size))
offset = numpy.random.randint(95, high = 106, size = (x_size, y_size)).astype(numpy.float)
read_noise = numpy.random.normal(loc = 1.5, scale = 0.1, size = (x_size, y_size))
c_data = [[0.0, "dark"], [1000.0, "light1"], [2000.0, "light2"], [3000, "light3"], [4000, "light4"]]
    
print("Mean offset", numpy.mean(offset))
print("Mean read noise", numpy.mean(read_noise))

def makeMovie(name, average_intensity):
    with tifffile.TiffWriter(name) as tf:
        for i in range(n_frames):
            # Signal - Poisson distribution
            image = gain * numpy.random.poisson(lam = average_intensity, size = (x_size, y_size))
        
            # Read Noise - Normal distribution
            image += numpy.random.normal(scale = read_noise, size = (x_size, y_size))
        
            # Camera baseline.        
            image += offset

            # 16 bit camera.
            tf.save(numpy.round(image).astype(numpy.uint16))
    print("Made", name)
        
for elt in c_data:
    makeMovie(elt[1] + ".tif", elt[0])
    

### Calibration

Compute statistics for each movie (mean and variance) and save in the appropriate format for `camera_calibration.py`.

In [ ]:
import storm_analysis.sCMOS.movie_to_calib_format as mTCF

for elt in c_data:
    bname = elt[1]
    [frame_mean, N, NN] = mTCF.movieToCalibration(bname + ".tif")
    mean = N/float(frame_mean.size)
    print("File", bname)
    print("Mean", numpy.mean(mean))
    print("Variance", numpy.mean(NN/float(frame_mean.size) - mean*mean))
    print("")
    numpy.save(bname + ".npy", [frame_mean, N, NN])

Run `camera_calibration.py` to calculate the sCMOS calibration file.

In [ ]:
import storm_analysis.sCMOS.camera_calibration as cam_cal

# Note: The statistics file should be in order from dimmest to brightest.
stat_files = list(map(lambda x: x[1] + ".npy", c_data))
print(stat_files)

# Note: The relative quantum efficiency is estimated by comparing the values
# in a smoothed version of the brightest image to the raw version.
#
[m_offset, m_variance, m_gain, m_rqe] = cam_cal.cameraCalibration(stat_files)

# The '2' is a file format marker. If this is left out then sCMOS analysis
# will interpret the data incorrectly.
#
# See the function loadSCMOS calibration in the file
# storm_analysis/sa_library/analysis_io.py for details.
#
numpy.save("calib.npy", [m_offset, m_variance, m_gain, m_rqe, 2])

print("Mean gain", numpy.mean(m_gain))

### Evaluate the results

How good a job did we do measuring the calibration?

In [ ]:
def makePlot(x, y, limits, xlabel, ylabel):
    fig = pyplot.figure(figsize = (8,8))
    ax = fig.add_subplot(1,1,1)
    ax.scatter(x, y, s = 4)
    ax.plot([limits[0],limits[1]],[limits[2],limits[3]], color = 'black', linewidth = 2)
    ax.axis("equal")
    ax.axis(limits)
    pyplot.xlabel(xlabel)
    pyplot.ylabel(ylabel)
    pyplot.show()

In [ ]:
import matplotlib
import matplotlib.pyplot as pyplot

import storm_analysis.sa_library.analysis_io as analysisIO

rn_sqr = read_noise * read_noise

# This is the function that analysis will use to load the sCMOS calibration data.
[f_offset, f_variance, f_gain, f_rqe] = analysisIO.loadCMOSCalibration("calib.npy", verbose = True)

print("Relative QE fractional measurement error: {0:.3f}".format(numpy.std(f_rqe)))

makePlot(offset.flatten(), f_offset.flatten(), [90, 110, 90, 110], "Offset", "Measured Offset")
makePlot(rn_sqr.flatten(), f_variance.flatten(), [1.25, 3.25, 1.25, 3.25], "Variance", "Measured Variance")
makePlot(gain.flatten(), f_gain.flatten(), [1.5, 2.5, 1.5, 2.5], "Gain", "Measured Gain")



### Also

The `storm-control` setup control software (available [here](https://github.com/ZhuangLab/storm-control)) can create the statistics files directly. This avoids having to store the possibly quite large movie files.